In [16]:
import ahk
import pyautogui
from time import sleep
import time
import win32gui
import win32api
import win32con
import PIL
from pynput.keyboard import Key, Listener 

import main
ahk = ahk.AHK()


def get_roblox_window():
    def callback(hwnd, extra):
        if win32gui.GetWindowText(hwnd) == "Roblox":
            extra.append(hwnd)

    windows = []
    win32gui.EnumWindows(callback, windows)
    return windows[0]

def get_current_active_window():
    # returns screen title
    return win32gui.GetWindowText(win32gui.GetForegroundWindow())


def activate_roblox():
    # if the window is showing but not active, win32gui.SetForegroundWindow(hwnd) will not work
    # so we need to click on the window to activate it
    hwnd = get_roblox_window()
    if win32gui.IsWindowVisible(hwnd) and not win32gui.IsIconic(hwnd):
        # then let's hide the window
        win32gui.ShowWindow(hwnd, win32con.SW_MINIMIZE)
        time.sleep(0.3)
    # then let's show the window and activate it
    win32gui.ShowWindow(hwnd, win32con.SW_MAXIMIZE)
    time.sleep(0.3)

# stop the script when press f1
def on_press(key):
    global is_running
    if key == Key.f1:
        is_running = False
        print("Stopping the script")
    
    if key == Key.f2:
        current_pos = pyautogui.position()
        current_position_color = pyautogui.pixel(current_pos[0], current_pos[1])
        print(f"Position: {current_pos}, Color: {current_position_color}")

listener = Listener(on_press=on_press)
listener.start()

def play_miniagme(game_duration=60):
    start_time = time.time()
    seek_pixel_position = (963, 596)
    seek_pixel_color = (168, 27, 14)
    drop_position = (975, 972)
    CLAIM_BUTTON_POSITION = (967, 666)

    is_running = True
    while is_running:
        pixel_color = pyautogui.pixel(*seek_pixel_position)
        if pixel_color != seek_pixel_color:
            ahk.mouse_move(*drop_position)
            ahk.click()
        if time.time() - start_time > game_duration:
            is_running = False
    
    # wait exit muinigame
    time.sleep(3)
    # now claim the reward
    ahk.mouse_move(*CLAIM_BUTTON_POSITION)
    ahk.click()
    time.sleep(0.3)


def get_pet_on_inventory_position(pet_row, pet_column):
    # Position: Point(x=584, y=311), Color: (228, 238, 240)
    PET_SLOT_WIDTH = 128
    PET_SLOT_HEIGHT = 128
    INVENTORY_START_POS = (584, 312)

    pet_x = INVENTORY_START_POS[0] + PET_SLOT_WIDTH * pet_column
    pet_y = INVENTORY_START_POS[1] + PET_SLOT_HEIGHT * pet_row

    # calculate the center of the pet slot
    pet_x += PET_SLOT_WIDTH // 2
    pet_y += PET_SLOT_HEIGHT // 2
    
    return (pet_x, pet_y)


def start_minigame(pet_row, pet_column):
    #Point(x=813, y=714), Color: (93, 255, 93)
    start_minigame_button_position = (813, 714)

    # open start mini game menu 
    ahk.key_down("e")
    ahk.key_up("e")
    time.sleep(0.5)

    # click on the start mini game button
    ahk.mouse_move(*start_minigame_button_position)
    ahk.click()
    time.sleep(0.3)

    # find the pet position on the inventory
    pet_position = get_pet_on_inventory_position(pet_row, pet_column)
    ahk.mouse_move(*pet_position)
    ahk.click()
    time.sleep(0.3)

    choose_button_image = PIL.Image.open("assets/choose_button.png")
    choose_button_position = pyautogui.locateCenterOnScreen(choose_button_image)

    ahk.mouse_move(choose_button_position[0], choose_button_position[1], relative=False, speed=10)
    ahk.click()

    time.sleep(0.3) # fade in animation

    # the minigame takes 3 seconds to start. let's wait for it
    time.sleep(3)
## vamos implementar a mesma funcao acima, mas desta vez calculando a posicao do pet

def restart_char():
    ahk.key_press("esc")
    time.sleep(0.3)
    ahk.key_press("r")
    time.sleep(0.3)
    ahk.key_press("enter")
    time.sleep(2)


def up_camera():
    # put the mouse on the center of the screen
    ahk.mouse_move(960, 540)
    time.sleep(0.3)
    
    # now, press right mouse button
    ahk.mouse_drag(x=960, y=560, from_position=(960, 540), speed=10, relative=False, button="right")

    # first, up all the way
    ahk.key_down("o")
    time.sleep(0.5)
    ahk.key_up("o")

    # then, down camera a little bit bc of floor 
    ahk.key_down("i")
    time.sleep(0.1)
    ahk.key_up("i")

def go_to_minigame():

    ahk.key_down("a")
    time.sleep(2.8)
    ahk.key_up("a")
    time.sleep(0.3)
    ahk.key_down("s")
    time.sleep(2.5)
    ahk.key_up("s")
    time.sleep(0.3)
    ahk.key_down("a")
    time.sleep(1.5)
    ahk.key_up("a")
    time.sleep(0.3)

In [17]:
main.run_boss()
activate_roblox()
restart_char()
up_camera()
go_to_minigame()
start_minigame(2, 2)
play_miniagme(60)

2024-04-06 00:59:12.824 | INFO     | main:run_boss:197 - Starting run 0
2024-04-06 00:59:20.803 | INFO     | main:freeze_game:165 - Freezing game for 48.93307605421687 seconds
2024-04-06 01:00:11.719 | INFO     | main:freeze_game:174 - Pixel color: (10, 8, 17)
2024-04-06 01:00:11.721 | INFO     | main:run_boss:216 - Waiting exit from boss room
2024-04-06 01:00:27.699 | INFO     | main:run_boss:224 - Run 1 took 74.87529230117798 seconds


Position: Point(x=967, y=666), Color: (112, 135, 165)
Position: Point(x=967, y=666), Color: (112, 135, 165)
Position: Point(x=967, y=666), Color: (112, 135, 165)
Position: Point(x=967, y=666), Color: (112, 135, 165)
Position: Point(x=967, y=666), Color: (112, 135, 165)
